In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#텍스트파일 읽기(*encoding='utf-8')/ 5~8:1, 1~4:0(평점레벨 1,0세그먼트)
df_train = pd.read_csv('data/감성분석/ratings_train.txt', 
                       sep='\t')
df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [7]:
df_train.shape

(150000, 3)

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [9]:
df_train.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [10]:
filt = df_train['document'].notnull()
df_train = df_train[filt]
df_train.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [ ]:
df_train.info() #null값 데이터 제외

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [ ]:
df_train['label'].value_counts() #0:부정/1:긍정

label
0    75170
1    74825
Name: count, dtype: int64

In [13]:
df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
#한글 외 영어/특수문자 등 제외-정규식사용 #^:연속>>데이터 전처리
import re
pattern = r'[^가-힣]+'
df_train['document']=df_train['document'].apply(lambda x:re.sub(pattern,' ',x)) #sub=replace

In [15]:
df_train.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [16]:
df_test = pd.read_csv('data/감성분석/ratings_test.txt', sep='\t')
df_test.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [17]:
df_test.isnull().sum()

id          0
document    3
label       0
dtype: int64

In [18]:
filt = df_test['document'].notnull()
df_test = df_test[filt]
df_test.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [19]:
pattern = r'[^가-힣]+'
df_test['document']=df_test['document'].apply(lambda x:re.sub(pattern,' ',x))

In [20]:
df_test.head()

,id,document,label
0,6270596,굳,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0


In [24]:
#전처리 데이터 저장하기
df_train.to_csv('data/감성분석/train.csv', index=False)

In [23]:
df_test.to_csv('data/감성분석/test.csv', index=False)

In [25]:
#영화 평점 코멘트에 대한 긍정/부정 판별 학습/ 텍스트 벡터화*
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
#테스트 텍스트
text = ['I go to my home my home is very large',
        'I went out my home I go to the market',
        'I bought a yellow lemon I go back to home']

In [39]:
tfidf = TfidfVectorizer(
    ngram_range=(1,2),
    min_df=2,
    max_df=0.9
) 
#vector 옵션-ngram_range:단어를 1 혹은 2개 조합으로 벡터화
#기본:단어1 min_df=2:2번이하반복단어제거 max_df=0.9 90%이상 단어 제거

tfidf.fit(text)
tfidf_text = tfidf.transform(text)
sorted(tfidf.vocabulary_.items()) #생성된 객체 확인

[('go to', 0), ('my', 1), ('my home', 2)]

In [40]:
#한글-형태소분리 konlpy
from konlpy.tag import Okt
okt = Okt()

In [41]:
#한글평태소분리 함수
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [42]:
text = '즐거운 추석 명절을 건강히 잘 보내세요'
okt_tokenizer(text)

['즐거운', '추석', '명절', '을', '건강히', '잘', '보내세요']

In [43]:
tfidf = TfidfVectorizer(
    tokenizer=okt_tokenizer, #한글형태소분리
    ngram_range=(1,2), #단어를 1 혹은 2개 조합으로 벡터화
    min_df=3, #2번이하반복단어제거
    max_df=0.9 #90%이상 반복 단어 제거
) 

In [59]:
df_train=df_train[:1000]
tfidf.fit(df_train['document'])
tfidf_train = tfidf.transform(df_train['document'])

c:\Users\AIPM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [60]:
sorted(tfidf.vocabulary_.items())

[('가', 0),
 ('가 너무', 1),
 ('가 더', 2),
 ('가 아닌', 3),
 ('가 이', 4),
 ('가슴', 5),
 ('가슴 이', 6),
 ('가요', 7),
 ('가장', 8),
 ('가족', 9),
 ('간', 10),
 ('간다', 11),
 ('갈수록', 12),
 ('감', 13),
 ('감독', 14),
 ('감독 님', 15),
 ('감독 의', 16),
 ('감독 이', 17),
 ('감동', 18),
 ('감동 도', 19),
 ('감동 을', 20),
 ('감동 적', 21),
 ('감정', 22),
 ('감정 을', 23),
 ('같다', 24),
 ('같아요', 25),
 ('같은', 26),
 ('같은 느낌', 27),
 ('같은 영화', 28),
 ('같은데', 29),
 ('같음', 30),
 ('개', 31),
 ('개봉', 32),
 ('개연', 33),
 ('개연 성', 34),
 ('개인', 35),
 ('개인 적', 36),
 ('걍', 37),
 ('거', 38),
 ('거기', 39),
 ('건', 40),
 ('건가', 41),
 ('건지', 42),
 ('걸', 43),
 ('겁나', 44),
 ('겁니다', 45),
 ('것', 46),
 ('것 도', 47),
 ('것 들', 48),
 ('것 은', 49),
 ('것 을', 50),
 ('것 이', 51),
 ('게', 52),
 ('결국', 53),
 ('결말', 54),
 ('계속', 55),
 ('고', 56),
 ('고추', 57),
 ('공감', 58),
 ('공부', 59),
 ('공포', 60),
 ('공포영화', 61),
 ('공포영화 는', 62),
 ('과', 63),
 ('관객', 64),
 ('괜찮은', 65),
 ('구', 66),
 ('구성', 67),
 ('국산', 68),
 ('굿', 69),
 ('귀신', 70),
 ('귀여운', 71),
 ('그', 72),
 ('그것', 73),
 ('그냥', 74),
 

In [61]:
tfidf_train.shape

(1000, 1002)

In [62]:
#로지스틱회귀모델 0/1 레벨 예측모델 생성
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=1, max_iter=500, random_state=0)

In [63]:
model.fit(tfidf_train, df_train['label'])

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,0
,solver,'lbfgs'
,max_iter,500
,multi_class,'deprecated'


In [64]:
#파라미터생성시 옵션-베스트모델 찾아주는 알고리즘
from sklearn.model_selection import GridSearchCV
params = {
    'C': [0.0001, 0.01, 0.1, 1, 10],
    'max_iter': [50,100,300,500]
}

grid = GridSearchCV(model, param_grid=params, scoring='accuracy', cv=3, verbose=1) 
#그룹단위:cv-3, verbose1과정표시

In [65]:
grid.fit(tfidf_train, df_train['label'])

Fitting 3 folds for each of 20 candidates, totalling 60 fits


,estimator,LogisticRegre...andom_state=0)
,param_grid,"{'C': [0.0001, 0.01, ...], 'max_iter': [50, 100, ...]}"
,scoring,'accuracy'
,n_jobs,None
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l2'


In [66]:
grid.best_params_, grid.best_score_

({'C': 1, 'max_iter': 50}, 0.6969814125502749)

In [67]:
best_model = grid.best_estimator_

In [68]:
train_pred = best_model.predict(tfidf_train)
train_pred[:50]

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0], dtype=int64)

In [69]:
df_train['prediction'] = train_pred

In [70]:
df_train.head()

,id,document,label,prediction
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1,1
2,10265843,너무재밓었다그래서보는것을추천한다,0,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,1


In [71]:
filt = df_train['label'] == df_train['prediction']
df_equal = df_train[filt]

In [72]:
len(df_train), len(df_equal), len(df_equal)/len(df_train)

(1000, 877, 0.877)

In [73]:
df_test.head()

,id,document,label
0,6270596,굳,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0


In [74]:
tfidf_test = tfidf.transform(df_test['document'])

In [75]:
test_pred = best_model.predict(tfidf_test)

In [76]:
df_test['prediction'] = test_pred
df_test.head()

,id,document,label,prediction
0,6270596,굳,1,0
1,9274899,,0,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,0


In [77]:
filt = df_test['label'] == df_test['prediction']
df_equal = df_test[filt]

In [78]:
len(df_test), len(df_equal), len(df_equal)/len(df_test)

(49997, 34718, 0.6944016640998459)

In [98]:
text = '영화가 너무 재밌어요. 주인공들도 연기를 참 잘해요.~~~'
find_text = re.findall(r'[가-힣]+', text)
text_join = [' '.join(find_text) ]#join, 배열>스트링
tfidf_text = tfidf.transform(text_join)
tfidf_text.shape

(1, 1002)

In [99]:
result = best_model.predict(tfidf_text)
if result[0]==1:
    print("긍정")
else:
    print("부정")

긍정


In [101]:
def predict(text):
    find_text = re.findall(r'[가-힣]+', text)
    text_join = [' '.join(find_text) ]#join, 배열>스트링
    tfidf_text = tfidf.transform(text_join)

    result = best_model.predict(tfidf_text)
    if result[0]==1:
        print("긍정")
    else:
        print("부정")

In [104]:
text = '영화가 너무 재밌어요. 주인공들도 연기를 참 잘해요.~~~'
predict(text)

긍정


In [ ]:
text = '지금까지 본 영화중 마음이 가장 따뜻해지는 영화'
predict(text)

긍정


In [107]:
text = '감동적이다....'
predict(text)

긍정
